In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [13]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 28, 28, 1).astype('float32') 
X_test = X_test.reshape(10000, 28, 28, 1).astype('float32')

X_train /= 255
X_test /= 255

n_classes = 10
y_train = to_categorical(y_train,n_classes)
y_test = to_categorical(y_test,n_classes)

random_seed = 3
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.1, random_state=random_seed)

In [14]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.3))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [15]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [16]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.3, 
                                            min_lr=0.00001)

In [18]:
epochs = 20
batch_size = 128

In [19]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # 使輸入數據的均值為0，這將使每個特徵在數據集上的均值為 0
        samplewise_center=False,  # 使輸入數據的每個樣本的均值為 0
        featurewise_std_normalization=False,  # 將輸入數據除以其標準差，以使每個特徵的方差為1
        samplewise_std_normalization=False,  # 將輸入數據的每個樣本除以其標準差
        zca_whitening=False,  # 應用ZCA白化
        rotation_range=10,  # 整數，隨機旋轉的度數範圍 (degrees, 0 to 180)
        zoom_range = 0.1, # 隨機縮放範圍 
        width_shift_range=0.1,  # 水平隨機移動的範圍 表示總寬度的比例 (fraction of total width)
        height_shift_range=0.1,  # 垂直隨機移動的範圍 表示總高度的比例 (fraction of total height)
        horizontal_flip=False,  # 隨機水平翻轉
        vertical_flip=False)  # 隨機垂直翻轉


datagen.fit(X_train)

In [20]:
history = model.fit(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_test,y_test),
                              verbose = 1, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

Epoch 1/20
421/421 [==============================] - 12s 27ms/step - loss: 0.3797 - accuracy: 0.8790 - val_loss: 0.0643 - val_accuracy: 0.9788
Epoch 2/20
421/421 [==============================] - 11s 27ms/step - loss: 0.1161 - accuracy: 0.9645 - val_loss: 0.0342 - val_accuracy: 0.9893
Epoch 3/20
421/421 [==============================] - 11s 27ms/step - loss: 0.0851 - accuracy: 0.9754 - val_loss: 0.0304 - val_accuracy: 0.9907
Epoch 4/20
421/421 [==============================] - 11s 26ms/step - loss: 0.0715 - accuracy: 0.9791 - val_loss: 0.0288 - val_accuracy: 0.9913
Epoch 5/20
421/421 [==============================] - 11s 27ms/step - loss: 0.0649 - accuracy: 0.9813 - val_loss: 0.0253 - val_accuracy: 0.9927
Epoch 6/20
421/421 [==============================] - 11s 27ms/step - loss: 0.0585 - accuracy: 0.9836 - val_loss: 0.0255 - val_accuracy: 0.9922
Epoch 7/20
421/421 [==============================] - 11s 27ms/step - loss: 0.0567 - accuracy: 0.9836 - val_loss: 0.0275 - val_accuracy:

In [24]:
model.save('CNN_Digit_Recognizer.h5')